In [ ]:
import glob
import pandas as pd
import cv2
import os
from shutil import copyfile
import numpy as np

# Libraries required for zooming
from scipy.ndimage import rotate
#from google.colab.patches import cv2_imshow
import numpy as np
from scipy.ndimage import zoom
import matplotlib.pyplot as plt

#!pip install split_folders
import split_folders

# Patchset to allow cv2.imshow
from google.colab.patches import cv2_imshow
#!pip install image_slicer
import image_slicer

In [ ]:
#choose path for image destination-validation images
root_path="/content/drive/My Drive/..........."
#choose path to obtain validation images
training_images_path = "/content/drive/My Drive.........."
#choose path to obtain csv file of annotations-validation images
annotated_data = "/content/drive/My Drive............"
annotated_data = pd.read_csv(annotated_data, header=0)

In [ ]:
def get_annotation(label_data, file):
     for rec in label_data.iterrows():
         if rec[1]['file_name'] == file:
             return rec[1]["annotation"]

In [ ]:
for image in os.listdir(training_images_path):
  if image.endswith(".jpg"):
    annotation = get_annotation(annotated_data, image)
    new_path = os.path.join(root_path, "classified_images", str(annotation))
    if not os.path.exists(new_path):
      os.makedirs(new_path)
    copyfile(os.path.join(training_images_path, image) , os.path.join(new_path, image))

KeyboardInterrupt: ignored

In [ ]:
# Checking number of images per folder

#validation data folder path
path="/content/drive/My Drive/..........."
for folder in os.listdir(path):
  print ("Class", folder, ":", len(os.listdir(os.path.join(path, folder))))

Class 1 : 96
Class 4 : 54
Class 0 : 133
Class 2 : 40
Class 3 : 39


In [ ]:
def clipped_zoom(img, zoom_factor, **kwargs):

    h, w = img.shape[:2]

    # For multichannel images we don't want to apply the zoom factor to the RGB
    # dimension, so instead we create a tuple of zoom factors, one per array
    # dimension, with 1's for any trailing dimensions after the width and height.
    zoom_tuple = (zoom_factor,) * 2 + (1,) * (img.ndim - 2)

    # Zooming out
    if zoom_factor < 1:

        # Bounding box of the zoomed-out image within the output array
        zh = int(np.round(h * zoom_factor))
        zw = int(np.round(w * zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        # Zero-padding
        out = np.zeros_like(img)
        out[top:top+zh, left:left+zw] = zoom(img, zoom_tuple, **kwargs)

    # Zooming in
    elif zoom_factor > 1:

        # Bounding box of the zoomed-in region within the input array
        zh = int(np.round(h / zoom_factor))
        zw = int(np.round(w / zoom_factor))
        top = (h - zh) // 2
        left = (w - zw) // 2

        out = zoom(img[top:top+zh, left:left+zw], zoom_tuple, **kwargs)

        # `out` might still be slightly larger than `img` due to rounding, so
        # trim off any extra pixels at the edges
        trim_top = ((out.shape[0] - h) // 2)
        trim_left = ((out.shape[1] - w) // 2)
        out = out[trim_top:trim_top+h, trim_left:trim_left+w]

    # If zoom_factor == 1, just return the input array
    else:
        out = img
    return out

In [ ]:
def increase_contrast(image):
    return cv2.convertScaleAbs(image, alpha = 1, beta = 50)
    

In [ ]:
def decrease_brightness(image):
    return cv2.convertScaleAbs(image, alpha = 0.7, beta = 1)

In [ ]:
def blur(image):
    return cv2.blur(image,(3,3))

In [ ]:
def rotate_zoom(image):
    rot = rotate(image, 20, reshape=False)
    return clipped_zoom(rot, 2)

In [ ]:
def rotate_zoom2(image):
    rot = rotate(image, 5, reshape=False)
    return clipped_zoom(rot, 2)

In [ ]:
def sharpen(image):
  kernel = np.array([[-1,-1,-1], [-1,9,-1], [-1,-1,-1]])
  return cv2.filter2D(image, -1, kernel)

In [ ]:
#root_path="/content/drive/My Drive/......"
!mkdir  root_path +"augmented_images/validation/0"
!mkdir  root_path+"augmented_images/validation/1"
!mkdir  root_path+"augmented_images/validation/2"
!mkdir  root_path+"augmented_images/validation/3"
!mkdir  root_path+"augmented_images/validation/4"

In [ ]:
#source path for validation images organized in folders according to categories0,1,2,3,4
src_path="/content/drive/My Drive/............."
#destination path for augmented validation images
dst_path="/content/drive/My Drive/............"

In [ ]:
DATA_PATH=src_path
AUG_PATH=dst_path

In [ ]:
category = "0"
i = 0
for f in os.listdir(DATA_PATH +  category):    
  if i % 8 == 0:
    slice(DATA_PATH +  category + "/" + f, DATA_PATH + category)  
  i = i + 1
i = 0

for f in os.listdir(DATA_PATH + category):  
  
  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)

  if i % 4 == 0:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    #cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       
    #cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 1:
       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", decrease_brightness(image))        
    #cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 2:

    #cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    #cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))        
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 3:

    #cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", #increase_contrast(image))        
    #cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", #sharpen(image))  
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))        

  i = i + 1


In [ ]:
category = "1"

for f in os.listdir(DATA_PATH +  category):    
  slice(DATA_PATH +  category + "/" + f, DATA_PATH + category)  
i = 0

for f in os.listdir(DATA_PATH + category):  
  
  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)

  if i % 4 == 0:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       

  elif i % 4 == 1:
       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 2:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))        

  elif i % 4 == 3:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))        
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  
    

  i = i + 1

 


In [ ]:
category = "2"
for f in os.listdir(DATA_PATH +  category):    
  slice(DATA_PATH +  category + "/" + f, DATA_PATH + category) 

for f in os.listdir(DATA_PATH + category):  

  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))   
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))    
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))        

In [ ]:
category = "3"
for f in os.listdir(DATA_PATH +  category):    
  slice(DATA_PATH +  category + "/" + f, DATA_PATH + category) 

for f in os.listdir(DATA_PATH + category):  

  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))   
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))    
  cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))        

In [ ]:
category = "4"
for f in os.listdir(DATA_PATH +  category):    
  slice(DATA_PATH +  category + "/" + f, DATA_PATH + category) 

i = 0

for f in os.listdir(DATA_PATH + category):  
  
  image = cv2.imread(DATA_PATH  + category + "/" + f)
  image = cv2.resize(image, (224, 224))
  f_name = f.split(".")[:-1][0]

  cv2.imwrite(AUG_PATH + category + "/" + f_name + ".jpg", image)

  if i % 4 == 0:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))   
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))        

  elif i % 4 == 1:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))       
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 2:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_b.jpg", decrease_brightness(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  

  elif i % 4 == 3:

    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_a.jpg", increase_contrast(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_c.jpg", blur(image))   
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_e.jpg", rotate_zoom2(image))    
    cv2.imwrite(AUG_PATH + category + "/" + f_name + "_f.jpg", sharpen(image))  
    

  i = i + 1

In [ ]:
for folder in os.listdir(AUG_PATH):
  print ("Class", folder, ":", len(os.listdir(os.path.join(AUG_PATH, folder))))

Class 0 : 266
Class 1 : 288
Class 2 : 240
Class 3 : 234
Class 4 : 270
